<a href="https://colab.research.google.com/github/archanaqre/Concurrent_ChatGPT_execution/blob/main/OpenAI_GPT_3_5_TURBO_EXECUTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install openai-multi-client

In [ ]:
import openai
import json
import csv
import requests

In [ ]:
import csv
import openai
from openai_multi_client import OpenAIMultiClient, Payload

openai.api_key = 'INSERT-API-KEY'  # Replace with your OpenAI API key

input_file_path = "INPUT-FILE-PATH.txt"  # Replace with the path to your input file
output_file_path = "oUTPUT-FILE-PATH.csv"  # Replace with the path to the output CSV file

api = OpenAIMultiClient(concurrency=15, max_retries=40, wait_interval=30, endpoint="chats", data_template={"model": "gpt-3.5-turbo"})


def on_result(result: Payload):
	line = result.metadata['line']
	response = result.response['choices'][0]['message']['content']  # Extract text from json outputs

	# Append the result to the output_data list
	output_data.append([line, response])
	write_output_to_csv()

def write_output_to_csv():
    with open(output_file_path, "a", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(output_data)

    # Clear the output_data list after writing to the CSV file
    output_data.clear()


def make_requests():
    with open(input_file_path, "r") as file:
        lines = file.readlines()

    for line in lines:
        line = line.strip("\n")
        messages = []
        messages = [{"role": "system", "content": "You are a translator assistant fluent in Malay-English codeswitched language"},
        	{"role": "user", "content": f"Given a codeswitched reference sentence '{line}' in Malay-English, generate 5 codeswitched variations of the sentence that are semantically equivalent to the original sentence. The variations should be of similar length to the original sentence."}]

        api.request(data={"messages": messages, "presence_penalty": -1.0}, metadata={'line': line}, callback=on_result)


output_data = []  # List to store the generated output

api.run_request_function(make_requests)
api.pull_all()

# Write any remaining output_data to the CSV file
write_output_to_csv()